In [10]:

from gpt4all import GPT4All
import pandas as pd
import requests
import json
import os
from datasets import load_dataset
import time

In [11]:
model = GPT4All("orca-mini-3b-gguf2-q4_0.gguf")


In [12]:
dataset = load_dataset("PatronusAI/financebench")


In [13]:
df = pd.DataFrame(dataset['train'])
df.head()

,financebench_id,doc_name,doc_link,doc_period,question_type,question,answer,evidence_text,page_number
0,financebench_id_03029,3M_2018_10K,https://investors.3m.com/financials/sec-filing...,2018,metrics-generated,What is the FY2018 capital expenditure amount ...,$1577.00,Table of Contents \n3M Company and Subsidiarie...,60
1,financebench_id_04672,3M_2018_10K,https://investors.3m.com/financials/sec-filing...,2018,metrics-generated,Assume that you are a public equities analyst....,$8.70,Table of Contents \n3M Company and Subsidiarie...,58
2,financebench_id_00499,3M_2022_10K,https://investors.3m.com/financials/sec-filing...,2022,domain-relevant,Is 3M a capital-intensive business based on FY...,"No, the company is managing its CAPEX and Fixe...",3M Company and Subsidiaries\n Consolidated Sta...,"48,50,52"
3,financebench_id_01226,3M_2022_10K,https://investors.3m.com/financials/sec-filing...,2022,domain-relevant,What drove operating margin change as of FY202...,Operating Margin for 3M in FY2022 has decrease...,"SG&A, measured as a percent of sales, increase...",27
4,financebench_id_01865,3M_2022_10K,https://investors.3m.com/financials/sec-filing...,2022,novel-generated,"If we exclude the impact of M&A, which segment...",The consumer segment shrunk by 0.9% organically.,Worldwide Sales Change\nBy Business Segment Or...,25


In [14]:
examples = [ f"<answer> {row['answer']} </answer>" for idx, row in df.iloc[:50].iterrows() ]
examples

['<answer> $1577.00 </answer>',
 '<answer> $8.70 </answer>',
 '<answer> No, the company is managing its CAPEX and Fixed Assets pretty efficiently, which is evident from below key metrics:\nCAPEX/Revenue Ratio: 5.1%\nFixed assets/Total Assets: 20%\nReturn on Assets= 12.4% </answer>',
 '<answer> Operating Margin for 3M in FY2022 has decreased by 1.7% primarily due to: \n-Decrease in gross Margin\n-mostly one-off charges including Combat Arms Earplugs litigation, impairment related to exiting PFAS manufacturing, costs related to exiting Russia and divestiture-related restructuring\ncharges </answer>',
 '<answer> The consumer segment shrunk by 0.9% organically. </answer>',
 "<answer> No. The quick ratio for 3M was 0.96 by Jun'23 close, which needs a bit of an improvement to touch the 1x mark </answer>",
 "<answer> Following debt securities registered under 3M's name are listed to trade on the New York Stock Exchange:\n-1.500% Notes due 2026 (Trading Symbol: MMM26)\n-1.750% Notes due 2030 (

In [15]:
# Placeholder prompts
HUMAN_PROMPT = "Hello! I'm seeking information about financial reports for a research project."
AI_PROMPT = " Please respond with concise, factual information based on the provided context and examples."

# List to store synthesized responses
syntheses_with_10_labels = []

# Loop through the DataFrame rows
start_time = time.time()
for idx, row in df.iterrows():
    question = row['question']
    evidence_text = row['evidence_text']

    # Define structured conversation template
    system_template = f"You are an accurate chatbot specialized in responding to queries about 10-K documents. Please only respond in the manner demonstrated by the following examples. If an explanation wasn't asked, refrain from providing one. "
    f"Based on the following examples: {examples[:10]}, please provide your answer in the same format. "
    f"please address the question: {question}. "
    f"Consider the provided text as evidence: {evidence_text}. "
    
    prompt_template = (
        f"USER: {question}\n"
        f"ASSISTANT: "
    )
    
    #initiate chat 
    with model.chat_session(system_template, prompt_template):
        response = model.generate(prompt=AI_PROMPT, temp=0.7)
        print(response)
        print(idx)      
        print("idx")
        end_time = time.time()  # End the timer
        print(f"Time taken for this idx: {end_time - start_time} seconds")
        syntheses_with_10_labels.append(response)

3M reported a total capital expenditure of $6,945 million in their fiscal year 2018 cash flow statement.
0
idx
Time taken for this idx: 60.2633330821991 seconds
3M Company's fiscal year ends on December 31st, which means their financial statements are as of and for the period from January 1st to December 31st. According to the company's balance sheet as of June 2018, the net profit for the period from January 1st to June 30th was $7.5 billion USD. Therefore, the year end FY2018 net PPNE for 3M is approximately $7.5 billion USD.
1
idx
Time taken for this idx: 142.03822588920593 seconds
3M is a capital-intensive business based on its financial statements for fiscal year 2022. The company invested heavily in research and development, which led to an increase in net debt.
2
idx
Time taken for this idx: 181.87700510025024 seconds
3M's operating margin change as of FY2022 can be attributed to various factors such as changes in sales volume, pricing strategy, cost structure, and product mix. 

In [ ]:
pd.DataFrame({'question': df['question'], 'answer' : df['answer'], 'evidence_text' : df['evidence_text'], 'syntheses' : syntheses_with_10_labels}).to_csv('syntheses_10.csv',index=False)


In [ ]:
# Placeholder prompts
HUMAN_PROMPT = "Hello! I'm seeking information about financial reports for a research project."
AI_PROMPT = " Please respond with concise, factual information based on the provided context and examples."

# List to store synthesized responses
syntheses_with_20_labels = []

# Loop through the DataFrame rows
start_time = time.time()
for idx, row in df.iterrows():
    question = row['question']
    evidence_text = row['evidence_text']

    # Define structured conversation template
    system_template = f"You are an accurate chatbot specialized in responding to queries about 10-K documents. Please only respond in the manner demonstrated by the following examples. If an explanation wasn't asked, refrain from providing one. "
    f"Based on the following examples: {examples[:20]}, please provide your answer in the same format. "
    f"please address the question: {question}. "
    f"Consider the provided text as evidence: {evidence_text}. "
    
    prompt_template = (
        f"USER: {question}\n"
        f"ASSISTANT: "
    )
    
    #initiate chat 
    with model.chat_session(system_template, prompt_template):
        response = model.generate(prompt=AI_PROMPT, temp=0.7)
        print(response)
        print(idx)      
        print("idx")
        end_time = time.time()  # End the timer
        print(f"Time taken for this idx: {end_time - start_time} seconds")
        syntheses_with_20_labels.append(response)

In [ ]:
pd.DataFrame({'question': df['question'], 'answer' : df['answer'], 'evidence_text' : df['evidence_text'], 'syntheses' : syntheses_with_20_labels}).to_csv('syntheses_20.csv',index=False)


In [ ]:
# Placeholder prompts
HUMAN_PROMPT = "Hello! I'm seeking information about financial reports for a research project."
AI_PROMPT = " Please respond with concise, factual information based on the provided context and examples."

syntheses_with_labels = []

for i in range(3, 6): 
    syntheses = []

    start_time = time.time()
    for idx, row in df.iterrows():
        question = row['question']
        evidence_text = row['evidence_text']
        

        # Define structured conversation template
        system_template = f"You are an accurate chatbot specialized in responding to queries about 10-K documents. Please only respond in the manner demonstrated by the following examples. If an explanation wasn't asked, refrain from providing one. "
        f"Based on the following examples: {examples[:i*10]}, please provide your answer in the same format. "
        f"please address the question: {question}. "
        f"Consider the provided text as evidence: {evidence_text}. "
        
    prompt_template = (
        f"USER: {question}\n"
        f"ASSISTANT: "
    )
    
   #initiate chat 
    with model.chat_session(system_template, prompt_template):
        response = model.generate(prompt=AI_PROMPT, temp=0.7)
        print(response)
        print(idx)      
        print("idx")
        end_time = time.time()  # End the timer
        print(f"Time taken for this idx: {end_time - start_time} seconds")
        syntheses.append(response)

    syntheses_with_labels.append(syntheses)  # Append all completions for the current i to syntheses_with_labels


In [ ]:
pd.DataFrame({'question': df['question'], 'answer' : df['answer'], 'evidence_text' : df['evidence_text'], 'syntheses' : syntheses_with_labels[0]}).to_csv('syntheses_30.csv',index=False)


In [ ]:
pd.DataFrame({'question': df['question'], 'answer' : df['answer'], 'evidence_text' : df['evidence_text'], 'syntheses' : syntheses_with_labels[1]}).to_csv('syntheses_40.csv',index=False)


In [ ]:
pd.DataFrame({'question': df['question'], 'answer' : df['answer'], 'evidence_text' : df['evidence_text'], 'syntheses' : syntheses_with_labels[2]}).to_csv('syntheses_50.csv',index=False)